In [5]:
import torch
import numpy

In [29]:
def parse_cfg():
    with open('config.txt', 'r') as file:
        lines = file.read().split('\n')                    # store the lines in a list
    lines = [x for x in lines if len(x) > 0]               # get read of the empty lines 
    lines = [x for x in lines if x[0] != '#']              # get rid of comments
    lines = [x.rstrip().lstrip() for x in lines]           # get rid of fringe whitespaces
    
    block = {}
    blocks = []
    
    for line in lines:
        if line[0] == "[":               # This marks the start of a new block
            if len(block) != 0:          # If block is not empty, implies it is storing values of previous block.
                blocks.append(block)     # add it the blocks list
                block = {}               # re-init the block
            block["type"] = line[1:-1].rstrip()     
        else:
            key,value = line.split("=") 
            block[key.rstrip()] = value.lstrip()
    blocks.append(block)
    
    return blocks

In [14]:
blocks = {
    'convolutional_1': {
        'type': 'convolutional',
        'batch_normalize': 1,
        'filters': 64,
        'size': 3,
        'stride': 2,
        'pad': 1,
        'activation': 'leaky'
    },
    'convolutional_2': {
        'type': 'convolutional',
        'batch_normalize': 1,
        'filters': 32,
        'size': 1,
        'stride': 1,
        'pad': 1,
        'activation': 'leaky'
    },
    'convolutional_3': {
        'type': 'convolutional',
        'batch_normalize': 1,
        'filters': 64,
        'size': 3,
        'stride': 1,
        'pad': 1,
        'activation': 'leaky'
    },
    'shortcut': {
        'type': 'shortcut',
        'from': -3,
        'activation': 'linear'
    },
    'upsample': {
        'type': 'upsample',
        'stride': 2
    },
    'route_1': {
        'type': 'route',
        'layers': [-4]
    },
    'route_2': {
        'type': 'route',
        'layers': [-1, 61]
    },
    'yolo': {
        'type': 'yolo',
        'mask': [0,1,2],
        'anchors': [[10,13],  [16,30],  [33,23],  [30,61],  [62,45],  [59,119],  [116,90],  [156,198],  [373,326]],
        'classes': 80,
        'num': 9,
        'jitter': 0.3,
        'ignore_thresh': 0.5,
        'truth_thresh': 1,
        'random': 1,
    },
    'net': {
        'type': 'net',
        # Testing
        'batch': 1,
        'subdivisions': 1,
        # Training
        # 'batch': 64,
        # 'subdivisions': 16,
        'width': 320,
        'height': 320,
        'channels': 3,
        'momentum': 0.9,
        'decay': 0.0005,
        'angle': 0,
        'saturation': 1.5,
        'exposure': 1.5,
        'hue': 0.1,
    },
}

In [23]:
class EmptyLayer(torch.nn.Module):
    def __init__(self):
        super(EmptyLayer, self).__init__()
        
class DetectionLayer(torch.nn.Module):
    def __init__(self, anchors):
        super(DetectionLayer, self).__init__()
        self.anchors = anchors

In [41]:
def create_modules(blocks):
    net_info = blocks[0] #  input and pre-processing information
    module_list = torch.nn.ModuleList()
    prev_filters = 3
    output_filters = []
    
    for index, block in enumerate(blocks[1:]):
        module = torch.nn.Sequential()
        
        # check the type of block
        # create a new module for the block
        # append to module_list
        
        # If it's a convolutional layer
        if block['type'] == 'convolutional':
            # Get the info about the layer
            activation = block['activation']
            try:
                batch_normalize = int(block['batch_normalize'])
                bias = False
            except:
                batch_normalize = 0
                bias = True
            filters = int(block['filters'])
            padding = int(block['pad'])
            kernel_size = int(block['size'])
            stride = int(block['stride'])
            if padding:
                pad = (kernel_size - 1) // 2
            else:
                pad = 0

            # Add the convolutional layer
            conv = torch.nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias = bias)
            module.add_module("conv_{0}".format(index), conv)

            # Add the Batch Norm Layer
            if batch_normalize:
                bn = torch.nn.BatchNorm2d(filters)
                module.add_module("batch_norm_{0}".format(index), bn)

            # Check the activation. 
            # It is either Linear or a Leaky ReLU for YOLO
            if activation == 'leaky':
                activn = torch.nn.LeakyReLU(0.1, inplace = True)
                module.add_module("leaky_{0}".format(index), activn)

        # If it's an upsampling layer
        # We use Bilinear2dUpsampling
        elif (block['type'] == 'upsample'):
            stride = int(block['stride'])
            upsample = torch.nn.Upsample(scale_factor = 2, mode = 'bilinear')
            module.add_module("upsample_{}".format(index), upsample)
            
        #If it is a route layer
        elif (block['type'] == 'route'):
            block['layers'] = block['layers'].split(',')
            #Start  of a route
            start = int(block['layers'][0])
            #end, if there exists one.
            try:
                end = int(block['layers'][1])
            except:
                end = 0
            #Positive anotation
            if start > 0: 
                start = start - index
            if end > 0:
                end = end - index
            route = EmptyLayer()
            module.add_module("route_{0}".format(index), route)
            if end < 0:
                filters = output_filters[index + start] + output_filters[index + end]
            else:
                filters= output_filters[index + start]

        #shortcut corresponds to skip connection
        elif block['type'] == 'shortcut':
            shortcut = EmptyLayer()
            module.add_module("shortcut_{}".format(index), shortcut)

        #Yolo is the detection layer
        elif block['type'] == 'yolo':
            mask = block['mask'].split(',')
            mask = [int(x) for x in mask]
    
            anchors = block['anchors'].split(',')
            anchors = [int(a) for a in anchors]
            anchors = [(anchors[i], anchors[i+1]) for i in range(0, len(anchors),2)]
            anchors = [anchors[i] for i in mask]
    
            detection = DetectionLayer(anchors)
            module.add_module("Detection_{}".format(index), detection)
                              
        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)
        
    return (net_info, module_list)

In [39]:
modules = create_modules(parse_cfg())
print(modules)
print(len(modules[0]))

([{'type': 'net', 'batch': '1', 'subdivisions': '1', 'width': '416', 'height': '416', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '32', 'size': '3', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '2', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '32', 'size': '1', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'shortcut', 'from': '-3', 'activation': 'linear'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '128', 'size

In [40]:
class Darknet(torch.nn.Module):
    def __init__(self):
        super(Darknet, self).__init__()
        self.blocks = parse_cfg()
        self.net_info, self.module_list = create_modules(self.blocks)
    def forward(self, x, CUDA):
        modules = self.blocks[1:]
        outputs = {} # We cache the outputs for the route layer
        write = 0     # This is explained a bit later
        for i, module in enumerate(modules):        
            module_type = (module["type"])
            if module_type == "convolutional" or module_type == "upsample":
                x = self.module_list[i](x)
            elif module_type == "route":
                layers = module["layers"]
                layers = [int(a) for a in layers]
                if (layers[0]) > 0:
                    layers[0] = layers[0] - i
                if len(layers) == 1:
                    x = outputs[i + (layers[0])]
                else:
                    if (layers[1]) > 0:
                        layers[1] = layers[1] - i
                    map1 = outputs[i + layers[0]]
                    map2 = outputs[i + layers[1]]
                    x = torch.cat((map1, map2), 1)
            elif  module_type == "shortcut":
                from_ = int(module["from"])
                x = outputs[i-1] + outputs[i+from_]
            elif module_type == 'yolo':        
                anchors = self.module_list[i][0].anchors
                #Get the input dimensions
                inp_dim = int (self.net_info["height"])
                #Get the number of classes
                num_classes = int (module["classes"])
                #Transform 
                x = x.data
                x = predict_transform(x, inp_dim, anchors, num_classes, CUDA)
                if not write:              #if no collector has been intialised. 
                    detections = x
                    write = 1
                else:       
                    detections = torch.cat((detections, x), 1)
            outputs[i] = x
        return detections